### Import

In [3]:
# import traci
import os, sys
#! default MacOS path of sumo tools
sys.path.append(os.path.join("/usr", "local", "opt", "sumo", "share", "sumo", "tools"))
# choose GUI or non GUI version
sumo_binary = "/usr/local/Cellar/sumo/1.15.0/bin/sumo"
#sumo_gui_binary = "/usr/local/Cellar/sumo/1.15.0/bin/sumo-gui"
#sumo_cmd = [sumo_binary, "-n", "/Users/jost/Workspace/traffic-simulation/xml/scenario1/grid.net.xml"]
sumo_cmd = [sumo_binary, "--start", "-c", "/Users/jost/Desktop/Magdeburgv2.sumocfg"]
import traci
from bs4 import BeautifulSoup
import numpy as np
import itertools

### Start SUMO / TRACI

In [4]:
# start traci
traci.start(sumo_cmd)

 Retrying in 1 seconds


(20, 'SUMO 1.15.0')

### Problems with traci
- `traci.lane.getFoes()` might use lane IDs twice for different connections
    - eg. junction at the FIN in Magdeburg

Solution: take net xml file, skip the ID matching and use foe matrix directly

In [5]:
with open("/Users/jost/2022-10-20-17-02-51/osm.net.xml", "r") as f:
    net_data = f.read()
net_xml = BeautifulSoup(net_data, "xml")

"""
for tls in traci.trafficlight.getIDList():
    get_eff_phases(tls)
"""

'\nfor tls in traci.trafficlight.getIDList():\n    get_eff_phases(tls)\n'

#### Method: `get_safe_phases(tls_id: str) -> total_safe_phases: list of lists`
- **INPUT:** tls ID as string
- **OUTPUT:** list of lists, where each inner lists contains connection indices that can share a green phase without colliding

**Concept:**  
Use the foe matrix of a nets xml file.  
A foe of a connection is another connection that might collide with it if they share the same green phase.  
  
Pairs of two connections sharing the same green phase without colliding (further named safe_phases or safe phase combinations) can be directly infered through the given foe matrix in a net xml file.  
Pairs of three, four, five and so on connections sharing the same green phase have to be computed based on the former safe phase combinations.  
  
The computation goes like:  
1. List pair of 2 connections using foe matrix
2. Intersect the safe connections of both connections in a pair of two to get the connections that are safe to combine with the pair
3. Combine all the intersected connections with the pair and do that for each pair to get pairs of 3 connections
4. For those pairs intersect the safe connections of all connections in the pair again to get further addable safe connections
5. Repeat the intersection and adding to combination steps until the intersections are empty i.e. there are no further combinable safe connections

In [6]:
# given a tls return all possible green combinations that are safe i.e. avoid collisions
#* INPUT: tls ID as string
#* OUTPUT: list of lists, where each inner lists means that given connection indices can share a green phase without colliding
#! make sure that indices are not changed during any parsing step
def get_safe_phases(tls_id: str):
    # extract junction ID from TLS ID -> ignore "GS_" at the beginning
    #! this might be net-specific
    junction_id = tls_id[3:]
    
    # whole junction including lanes etc
    junction_data = net_xml.find('junction', {'id': junction_id})
    # just request data including foes, cont, index, response
    request_data = junction_data.find_all('request')
    # just foes
    foes = [request.attrs['foes'] for request in request_data]
    # foes as np array
    foes = np.ma.array([list(map(int, foe)) for foe in foes], mask=False)
    
    # number of tls / tls controlled connections
    n_tls = foes.shape[0]
    
    #! flip foes to be more intuitive, without flipping columns would be reversed i.e. last value in each row would correspond to index 0
    foes = np.fliplr(foes)
    
    # mask values corresponding to whether a connection has itself as foe or not
    # we want to ignore those for further computations
    for i in range(n_tls):
        foes.mask[i][i] = True
    
    # non-foes as a non sparse non binary list of foe indices
    # practically list of connection indices that don't collide i.e. that can share a green phase
    non_foes_ind = [np.where(foes[i] == 0)[0] for i in range(n_tls)]
    
    # placeholder to collect all safe_phases (safe phase combinations) during computation
    total_safe_phases = []
    
    # for each tls / tls controlled connection list all others that are no foes of each other
    # this combines all safe connection combinations which are directly readable from the foe matrix and which don't need further computation
    safe_phases = []
    for i in range(n_tls):
        # pairs of 2: extract from foes
        for i2 in non_foes_ind[i]:
            #* condition to not take duplicates e.g. [0, 1] and [1, 0]
            if i2 > i:
                safe_phases.append([i, i2])
            else:
                continue
    
    # add safe phases to collection
    total_safe_phases += safe_phases
    
    # placeholder for computation of next level non-foes
    curr_non_foes = []
    
    # for each safe_phase that was combined in the first step -> get the safe connections of the given safe_phase and intersect those safe connections with the possible safe connections of an connection added to the safe phase
    # the resulting safe phases contain more connections that can go green in parallel but also leads to less further possibilities to add to that combination
    for safe_phase in safe_phases:
        # start with first connection of a safe phase
        wip_non_foes = non_foes_ind[safe_phase[0]]
        # for each other connection of a safe phase get the step-wise intersection
        for connection in safe_phase[1:]:
            wip_non_foes = np.intersect1d(wip_non_foes, non_foes_ind[connection])
        curr_non_foes.append(wip_non_foes)
        
    # check if non foes exist
    # this is used as end condition for the loop
    non_foes_exist = False
    for non_foe in curr_non_foes:
        if non_foe.any():
            non_foes_exist = True
            break
    
    # iteratively look for safe_phases and further non_foes
    # stop when there are no further connection combinations having no_foe options to add
    while non_foes_exist: 
        # get new safe_phases
        #! duplicates allowed, filter befor appending
        dupl_safe_phases = []
        for i in range(len(safe_phases)):
            for connection in curr_non_foes[i]:
                dupl_safe_phases.append(safe_phases[i].copy() + [connection])
        # filter duplicates
        for safe_phase in dupl_safe_phases:
            safe_phase.sort()
        dupl_safe_phases.sort()
        # these are the newly computed safe phase combinations
        new_safe_phases = list(k for k,_ in itertools.groupby(dupl_safe_phases))
    
        # get new indices of non_foes for the newly computed safe phase combinations
        # so to speak: for each safe combinations a list of connection indices that could further be added to a given combination
        new_non_foes = []
        for i in range(len(new_safe_phases)):
            wip_non_foes = non_foes_ind[new_safe_phases[i][0]]
            for connection in new_safe_phases[i][1:]:
                wip_non_foes = np.intersect1d(wip_non_foes, non_foes_ind[connection])
            new_non_foes.append(wip_non_foes)
        
        # add safe phases to collection
        total_safe_phases += new_safe_phases
        
        # overwrite old values for next iteration
        curr_non_foes = new_non_foes.copy()
        # check if further non foes exist and set ending condition accordingly
        non_foes_exist = False
        for non_foe in curr_non_foes:
            if non_foe.any():
                non_foes_exist = True
                break
                
        safe_phases = new_safe_phases.copy()
    
    # return collection of all connection combinations that can share a green phase and would not lead to collisions
    # list of lists where each inner list contains indices of connections
    return total_safe_phases

In [7]:
all_safe_phases = get_safe_phases("GS_cluster_244996795_33401116_4471941369_60603339")

print(all_safe_phases)

[[0, 1], [0, 2], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [1, 2], [1, 6], [1, 7], [1, 8], [2, 3], [2, 4], [2, 5], [2, 6], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [3, 4], [3, 5], [3, 6], [3, 10], [3, 11], [3, 12], [3, 13], [4, 5], [4, 6], [4, 10], [4, 11], [4, 12], [4, 13], [5, 6], [6, 7], [6, 8], [6, 9], [6, 10], [6, 11], [7, 8], [7, 9], [7, 10], [7, 11], [8, 9], [8, 10], [8, 11], [9, 10], [9, 11], [10, 12], [10, 13], [11, 12], [11, 13], [12, 13], [0, 1, 2], [0, 1, 6], [0, 1, 7], [0, 1, 8], [0, 2, 5], [0, 2, 6], [0, 2, 9], [0, 2, 10], [0, 2, 11], [0, 2, 12], [0, 2, 13], [0, 5, 6], [0, 6, 7], [0, 6, 8], [0, 6, 9], [0, 6, 10], [0, 6, 11], [0, 7, 8], [0, 7, 9], [0, 7, 10], [0, 7, 11], [0, 8, 9], [0, 8, 10], [0, 8, 11], [0, 9, 10], [0, 9, 11], [0, 10, 12], [0, 10, 13], [0, 11, 12], [0, 11, 13], [0, 12, 13], [1, 2, 6], [1, 6, 7], [1, 6, 8], [1, 7, 8], [2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 10], [2, 3, 11], [2, 3, 12], [2, 3, 13], [2, 4, 5], [2, 4, 6],

### Close SUMO / TRACI

In [8]:
# end
traci.close()